In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)
!pip install sentencepiece
import nltk
nltk.download('stopwords')

TRAINING

In [ ]:
%%shell
SUFFIX="key_word_multiplied_3_luong_3_layer_200_neurons"
FIRST_ATTEMPT=1
LAST_ATTEMPT=5
SAVE_DIR=$(echo "drive/My Drive/masters/"$SUFFIX"/save")
LOG_DIR=$(echo "drive/My Drive/masters/"$SUFFIX"/logs")
mkdir -p "drive/My Drive/masters/"$SUFFIX"/generated"
COMMAND="python3 'drive/My Drive/masters/train.py' --key_word_count_multiplier 3 --rnn_size 200 --num_layers 3 --attention_type luong --num_epochs 100 --save_every 1 --gpu_mem 0.95 --vocab_size 10000 --use_attention --unk_max_count 100 --unk_max_number 1 --data_dir 'drive/My Drive/masters/data/training_data' --validation_data_dir 'drive/My Drive/masters/data/validation_data'"

for attempt in `seq -s ' ' $FIRST_ATTEMPT $LAST_ATTEMPT`; do

    ATTEMPT_SAVE_DIR=$(echo "$SAVE_DIR/attempt_$attempt")
    ATTEMPT_LOG_DIR=$(echo "$LOG_DIR/attempt_$attempt")
    rm -rf "$ATTEMPT_SAVE_DIR" "$ATTEMPT_LOG_DIR"
    mkdir -p "$ATTEMPT_SAVE_DIR" "$ATTEMPT_LOG_DIR"
    command=$(echo "$COMMAND --log_dir '$ATTEMPT_LOG_DIR' --save_dir '$ATTEMPT_SAVE_DIR'")
    eval $command
    echo $attempt

done

reading text file
Initial data size: 323748
Size after removing utterances without specified pos tags: 323748
Size after removing utterances longer than seq_length: 321936
Size after removing utterances with wrong number of key words: 320784
Size after removing utterances not meeting unk_max_number criteria: 314569
Size after removing utterances not meeting unk_max_count criteria: 285661
training data loaded!
reading text file
Initial data size: 35973
Size after removing utterances without specified pos tags: 35973
Size after removing utterances longer than seq_length: 35770
Size after removing utterances with wrong number of key words: 35627
Size after removing utterances not meeting unk_max_number criteria: 34821
Size after removing utterances not meeting unk_max_count criteria: 31419
validation data loaded!
W0818 23:02:58.420528 140080546387840 deprecation.py:323] From /content/drive/My Drive/masters/model.py:34: BasicLSTMCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is d

TESTING

In [ ]:
%%shell
FOLDER="key_word_multiplied_3_luong_3_layer_200_neurons"
GENERATED_DIR=$(echo "drive/My Drive/masters/generated/$FOLDER")
LOG_DIR=$(echo "drive/My Drive/masters/$FOLDER/logs")
FULL_LOG_DIR=$(echo "drive/My Drive/masters/logs/$FOLDER")
BEST_EPOCHS_FILE=$(echo "$FULL_LOG_DIR/best_epochs.txt")
mkdir -p "'$GENERATED_DIR'" "'$FULL_LOG_DIR'"
rm -f "$BEST_EPOCHS_FILE"

for attempt in `seq -s ' ' 1 5`; do

    ATTEMPT_FILE=$(echo "$LOG_DIR/attempt_$attempt/validation_log.txt")
    ATTEMPT_LOG_DIR=$(echo "$FULL_LOG_DIR/attempt_$attempt")
    mkdir -p "$ATTEMPT_LOG_DIR"
    cp "$ATTEMPT_FILE" "$ATTEMPT_LOG_DIR"
    echo $attempt
    grep -v ^$ "$ATTEMPT_FILE" | sort -t $'\t' -V -k4,4 | head -n 1
    echo $attempt >> "$BEST_EPOCHS_FILE"
    grep -v ^$ "$ATTEMPT_FILE" | sort -t $'\t' -V -k4,4 | head -n 1 >> "$BEST_EPOCHS_FILE"
done

1
epoch	5	validation_loss	14.953	validation_time	23.820
2
epoch	6	validation_loss	15.176	validation_time	23.927
3
epoch	8	validation_loss	15.059	validation_time	24.820
4
epoch	8	validation_loss	15.130	validation_time	24.073
5
epoch	8	validation_loss	15.182	validation_time	21.601


GENERATION

In [ ]:
%%shell
FOLDER="key_word_multiplied_3_luong_3_layer_200_neurons"
ATTEMPT=1
SAVE_DIR=$(echo "drive/My Drive/masters/$FOLDER/save/attempt_$ATTEMPT")
NR_OF_LINES=$(wc -l < "drive/My Drive/masters/data/test_data/input.txt")
CHUNK_SIZE=100
for state in `echo "random"`; do
  PICK=2
  SAMPLE=1
  WIDTH=4
  GENERATED_DIR=$(echo "drive/My Drive/masters/generated_best/width_${WIDTH}_pick_${PICK}_sample_${SAMPLE}")
  mkdir -p "'$GENERATED_DIR'"
  COMMAND="python3 'drive/My Drive/masters/generate_utterances.py' --state_initialization $state -i '$SAVE_DIR' -t 'drive/My Drive/masters/data/test_data/input.txt' --width=$WIDTH --sample=$SAMPLE --pick=$PICK --tokens -q -o '$GENERATED_DIR' -k 'drive/My Drive/masters/data/test_data/input_tagged.txt'"
  
  for start_index in `seq -s ' ' 0 $CHUNK_SIZE $NR_OF_LINES`; do

      command=$(echo "$COMMAND --start_index $start_index --nr_of_loops $CHUNK_SIZE")
      eval $command

  done
done